# 1.Import libraries :

In [1]:
!pip install --upgrade pip
!pip install yt-dlp  moviepy gradio langchain langchain-openai faiss-cpu langsmith
!pip install -U langchain-community
!pip install -U langchain
!pip install gradio pyngrok
!pip install gtts
!pip install SpeechRecognition pydub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 79.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 125.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 147.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 140.0 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.56
    Uninstalling langchain-core-0.3.56:
      Successfully uninstalled langchain-core-0.3.56
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19/19 [langchain-openai]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 104.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8/8 [lan

In [2]:
!pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-91x2zw3n
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-91x2zw3n
  Resolved https://github.com/openai/whisper.git to commit 517a43ecd132a2089d85f4ebc044728a71d49f6e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 168.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 164.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 76.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 154.1 MB/s eta 0:00:00
 

# 2.all the codes in one cell:

In [4]:
import os
import re
import time
import random
import tempfile
import json
import yt_dlp
import whisper
import speech_recognition as sr
from pydub import AudioSegment
from moviepy.editor import VideoFileClip
import gradio as gr
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA
from langchain_core.output_parsers import StrOutputParser
from langchain.agents import initialize_agent, AgentType, tool
from langchain_core.tools import Tool
from gtts import gTTS
from graphviz import Digraph
import concurrent.futures  # for parallelization

# Environment setup
# openai_api_key = os.getenv("OPENAI_API_KEY")
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_PROJECT"] = "youtube-video-summarizer"
import os
import openai
# Replace with your real keys
os.environ["OPENAI_API_KEY"] = "sk-proj-my key"
os.environ["LANGCHAIN_API_KEY"] = "my key"
os.environ["LANGCHAIN_PROJECT"] = "youtube-rag-bot"
openai.api_key = "my key"

whisper_model = whisper.load_model("base")

PROMPT = PromptTemplate(
    input_variables=["question", "context"],
    template="""Use the transcript below to answer the question. Respond in bullet points if possible.\nTranscript:\n{context}\nQuestion:\n{question}"""
)

# Helpers
def is_valid_youtube_url(url):
    pattern = r"(https?://)?(www\.)?(youtube\.com|youtu\.be)/.+"  # regex to check for valid youtube url
    return re.match(pattern, url)

def download_youtube_video(url):
    if not is_valid_youtube_url(url):
        return {"error": "Invalid YouTube URL."}

    delay = random.uniform(4, 6)  # Adjusted sleep delay for bot detection avoidance
    print(f"Sleeping for {delay:.2f} seconds to avoid bot detection...")
    time.sleep(delay)

    temp_dir = tempfile.mkdtemp()
    output_path = os.path.join(temp_dir, "%(title)s.%(ext)s")

    ydl_opts = {
        'format': 'bestvideo+bestaudio/best',
        'outtmpl': output_path,
        'quiet': True,
        'merge_output_format': 'mp4',
        'noplaylist': True,
        'ratelimit': 512 * 1024,
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(url, download=True)
            filename = ydl.prepare_filename(info).replace(".webm", ".mp4")
            metadata = {
                "title": info.get("title"),
                "uploader": info.get("uploader"),
                "duration": round(info.get("duration", 0) / 60, 2),
                "description": info.get("description", "")[:500],
                "filename": filename
            }

        time.sleep(random.uniform(2, 3))  # Shortened delay after download
        return metadata

    except Exception as e:
        return {"error": str(e)}

def transcribe_video(file_path):
    time.sleep(random.uniform(2, 4))  # Sleep added for transcription process to mitigate strain
    try:
        audio_path = file_path.replace(".mp4", ".wav")
        video_clip = VideoFileClip(file_path)
        if video_clip.audio is None:
            return {"error": "No audio track found in video."}
        video_clip.audio.write_audiofile(audio_path, verbose=False, logger=None)
        result = whisper_model.transcribe(audio_path, word_timestamps=True)
        os.remove(audio_path)
        return result
    except Exception as e:
        return {"error": f"Error during transcription: {str(e)}"}


def split_transcript_chunks(transcript):
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    return splitter.split_text(transcript)

def build_vector_db(chunks):
    #embeddings = OpenAIEmbeddings(api_key=openai_api_key)
    openai_api_key = os.getenv("OPENAI_API_KEY")  # Ensure your API key is set in Colab
    embeddings = OpenAIEmbeddings(api_key=openai_api_key)

    return FAISS.from_texts(chunks, embedding=embeddings)

def create_qa_chain(vector_db):
    llm = ChatOpenAI(model_name="gpt-4", temperature=0.3, api_key=openai_api_key)
    return RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=vector_db.as_retriever(),
        input_key="query",
        chain_type_kwargs={"prompt": PROMPT}
    )

def summarize_with_langsmith(transcript):
    prompt = ChatPromptTemplate.from_template(
        "You are a helpful assistant. Summarize the following YouTube video transcript in bullet points:\n\n{transcript}"
    )
    llm = ChatOpenAI(model_name="gpt-4", temperature=0.3, api_key=openai_api_key)
    chain = prompt | llm | StrOutputParser()
    return chain.invoke({"transcript": transcript})


def text_to_speech(text):
    tts = gTTS(text)
    audio_file = tempfile.mktemp(suffix=".mp3")
    tts.save(audio_file)
    return audio_file

def speech_to_text(audio_file_path):
    recognizer = sr.Recognizer()
    audio = AudioSegment.from_file(audio_file_path)
    wav_path = tempfile.mktemp(suffix=".wav")
    audio.export(wav_path, format="wav")
    try:
        with sr.AudioFile(wav_path) as source:
            audio_data = recognizer.record(source)
            return recognizer.recognize_google(audio_data)
    except sr.UnknownValueError:
        return {"error": "Speech recognition could not understand the audio."}
    except sr.RequestError as e:
        return {"error": f"Speech recognition request failed: {str(e)}"}
    except Exception as e:
        return {"error": f"Unexpected error in speech-to-text: {str(e)}"}


def generate_mind_map_structure(summary_text):
    prompt = ChatPromptTemplate.from_template("""
    Convert the following summary into a detailed mind map structure. Return JSON with:
    - 'main_topic': the central theme
    - 'branches': list of dictionaries with 'title', 'subpoints' (bullets), and optionally 'examples'.

    Summary:
    {summary}
    """)
    llm = ChatOpenAI(model_name="gpt-4", temperature=0.3, api_key=openai_api_key)
    chain = prompt | llm | StrOutputParser()
    response = chain.invoke({"summary": summary_text})
    try:
        return json.loads(response)
    except json.JSONDecodeError as e:
        print("Mind map JSON error:", response)
        raise e


def create_mind_map(summary_text):
    try:
        structured_data = generate_mind_map_structure(summary_text)
        dot = Digraph(comment="Mind Map")
        dot.attr(rankdir='LR', fontsize='12', fontname='Helvetica')

        dot.node("0", structured_data["main_topic"], shape='box', style='filled', fillcolor='lightblue')

        for i, branch in enumerate(structured_data["branches"], 1):
            parent_id = f"{i}"
            dot.node(parent_id, branch["title"], shape='ellipse', style='filled', fillcolor='lightyellow')
            dot.edge("0", parent_id)

            for j, subpoint in enumerate(branch.get("subpoints", []), 1):
                child_id = f"{i}.{j}"
                wrapped = "\n".join([subpoint[k:k+50] for k in range(0, len(subpoint), 50)])
                dot.node(child_id, wrapped, shape='note', fontsize='10')
                dot.edge(parent_id, child_id)

        output_path = tempfile.mktemp(suffix=".png")
        dot.render(filename=output_path, format='png', cleanup=True)
        return output_path + ".png"
    except Exception as e:
        return {"error": f"Error generating mind map: {str(e)}"}

# === LangChain Agent Tools and Agent Setup ===

@tool
def summarize_transcript_tool(transcript: str) -> str:
    """Summarize a transcript using GPT-4."""
    return summarize_with_langsmith(transcript)

@tool
def ask_question_tool(context: str, question: str) -> str:
    """Answer a question based on the transcript."""
    chunks = split_transcript_chunks(context)
    vector_db = build_vector_db(chunks)
    qa_chain = create_qa_chain(vector_db)
    return qa_chain.run(question)

openai_api_key = os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI(model="gpt-4", temperature=0.3, api_key=openai_api_key)

tools = [
    Tool.from_function(
        summarize_transcript_tool,
        name="summarize_transcript_tool",
        description="Summarize a transcript using GPT-4"
    ),
    Tool.from_function(
        ask_question_tool,
        name="ask_question_tool",
        description="Answer a question based on the transcript context"
    )
]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# === Main logic function with parallelization ===

def handle_video_task(url, question, task, voice_question_audio=None):
    if not url or not is_valid_youtube_url(url):
        return "Invalid URL", "", "", "", "", None, None

    if voice_question_audio and not question:
        question = speech_to_text(voice_question_audio)
        if "error" in question:  # if speech-to-text fails
            return question["error"], "", "", "", "", None, None

    metadata = download_youtube_video(url)
    if "error" in metadata:
        return metadata["error"], "", "", "", "", None, None

    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_transcription = executor.submit(transcribe_video, metadata["filename"])
        transcript_result = future_transcription.result()

        if "error" in transcript_result:
            return transcript_result["error"], "", "", "", "", None, None

        transcript = transcript_result["text"]
        summary = ""
        audio_path = None
        mind_map_path = None

        try:
               if task == "ask_question" and question:
                    summary = agent.invoke(f"Use ask_question_tool to answer this question: '{question}' using this context: {transcript}")
               elif task == "langsmith_summary":
                    summary = agent.invoke(f"Use summarize_transcript_tool to summarize this transcript: {transcript}")

               if isinstance(summary, dict) and "output" in summary:
                    summary = summary["output"]
        except Exception as e:
            return {"error": f"Error while processing task: {str(e)}"}, "", "", "", "", None, None

        if summary:
            audio_path = text_to_speech(summary)
            mind_map_path = create_mind_map(summary)

    return transcript, summary, metadata["title"], metadata["uploader"], f"{metadata['duration']} min", audio_path, mind_map_path

# === Gradio UI ===
# (UI code remains the same)

with gr.Blocks(css="""
    .small-audio {
        max-width: 100% !important;
        height: 150px !important;
        min-width: 320px !important;
    }
    .gradio-container {
        background-color: #f5f9fd;
        font-family: 'Helvetica Neue', sans-serif;
    }
    #main-title {
        color: #92b8dd;
        font-size: 28px;
        text-align: center;
        margin-bottom: 10px;
    }
    .section-box {
        border: 1px solid #d1e3f3;
        padding: 16px;
        border-radius: 10px;
        background-color: #ffffff;
    }
    .submit-button {
        background-color: #92b8dd !important;
        color: white !important;
        font-weight: bold;
    }
""") as demo:

    #gr.Markdown("## 🎥 YouTube Video AI Assistant", elem_id="main-title")
    gr.Markdown("## 🎥NeuroTube AI Assistant", elem_id="main-title")


    with gr.Tab("🎬 Main"):
        with gr.Row(equal_height=True):
            with gr.Column(scale=3, elem_classes="section-box"):
                youtube_input = gr.Textbox(label="🔗 YouTube URL", placeholder="Paste YouTube video link")

                with gr.Row():
                    question_input = gr.Textbox(label="Text Question", placeholder="Type your question")
                    voice_input = gr.Audio(label="Voice", type="filepath", elem_classes="small-audio")

                task_choice = gr.Radio(
                    ["ask_question", "langsmith_summary"],
                    label="Choose a Task",
                    value="langsmith_summary"
                )

                submit_btn = gr.Button("🚀 Run", elem_classes="submit-button")

            with gr.Column(scale=2, elem_classes="section-box"):
                gr.Markdown("### 📺 Video Info")
                title_output = gr.Textbox(label="Title", interactive=False)
                uploader_output = gr.Textbox(label="Uploader", interactive=False)
                duration_output = gr.Textbox(label="Duration", interactive=False)

        with gr.Row():
            with gr.Column(elem_classes="section-box"):
                gr.Markdown("### 🧠 AI Summary / Answer")
                response_output = gr.Textbox(label="", lines=6)

            with gr.Column(elem_classes="section-box"):
                gr.Markdown("### 🔊 Listen to the Answer")
                audio_output = gr.Audio(label="AI Voice")

    with gr.Tab("🗺 Mind Map"):
        gr.Markdown("### Visual Mind Map")
        mindmap_output = gr.Image(label="Concept Map")

    with gr.Tab("📄 Transcript"):
        gr.Markdown("### Full Video Transcript")
        transcript_output = gr.Textbox(label="", lines=25)

    submit_btn.click(
        handle_video_task,
        inputs=[youtube_input, question_input, task_choice, voice_input],
        outputs=[
            transcript_output, response_output,
            title_output, uploader_output, duration_output,
            audio_output, mindmap_output
        ]
    )

if __name__ == "__main__":
    demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://597a98cda007de0293.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
